# 2. Feature Engineering and Modifying Data For Prediction Model

저는 해당 노래를 다시 들을지에 대한 확율이 그 고객의 취향과 개인 문화권, 노래의 대중성에 달렸다고 생각을 하였습니다.

고객의 취향
- train 데이터에서 고객이 'suger'이라는 노래를 들 었다고 할 떄, 그 노래에 해당하는 작곡가, 작사가, 장르, 혹은 가수가 같은 노래를 몇 번이나 들었는지, 그리고 각 작곡가, 작사가, 장르, 가수에 따른 평균 target 값이 몇 인지를 구하여 고객의 데이터에 추가 하였습니다

고객의 문화권
- 사는 도시, 사용 언어, 서비스 접속 도구, 성별을 카테고리 변수로 만들어 더했습니다

대중성
- 해당 노래를 사람들이 몇 번이나 들었는지, 각각의 작곡가, 작사가, 장르, 그리고 가수들이 포함되는 노래가 데이터 세트에 몇 개나 존재하는지를 카운트 하여 데이터 세트에 더했습니다


In [1]:
import pandas as pd
import numpy as np

### Compare Train Data Set and Test Data Set

In [ ]:
test = pd.read_csv('test.csv', encoding = 'utf-8')
train = pd.read_csv('train.csv', encoding = 'utf-8')

In [ ]:
songs = pd.read_csv('songs.csv', encoding = 'utf-8')
members = pd.read_csv('members.csv', encoding = 'utf-8')

In [ ]:
data_sum = [test, train]
for df in data_sum:
    df1 = pd.merge(df, songs, on='song_id')
    df2 = pd.merge(df1, members, on='msno')
#    del df2['language']
    if len(df2) > 7000000:
        train = df2
    else:
        test = df2


In [ ]:
def compare_test_train(col):
    set_train = set(train[col])
    set_test = set(test[col])
    i = len(set_test - set_train -set([None]))
    print('number of test set  %s, not in train set' % col, ':', i)
    
def compare_train_test(col):
    set_train = set(train[col])
    set_test = set(test[col])
    i = len(set_train - set_test -set([None]))
    print('number of train set  %s, not in test set' % col, ':', i)

In [ ]:
compare_train_test('language')
compare_test_train('language')

In [ ]:
lis = ['source_system_tab', 'source_screen_name', 'source_type', 'genre_ids', 'composer', 'lyricist', 'gender', 'language']
lis1 = ['no_tab', 'no_name', 'no_type', 'no_genre', 'no_composer', 'no_lyricist', 'no_gender', 'no_language']
for x, y in zip(lis, lis1):
    train[x] = train[x].fillna(y)
    test[x] = test[x].fillna(y)

In [ ]:
category_var = ['source_system_tab', 'source_screen_name', 'source_type', 'city', 'gender', 'registered_via', 'language']

for var in category_var:
    compare_train_test(var)
for var in category_var:
    compare_test_train(var)
    

In [ ]:
SCN = set(test['source_screen_name']) - set(train['source_screen_name'])
RV = set(test['registered_via']) - set(train['registered_via'])
print('Source screen name not in train set : ' , SCN)
print('registered_via name not in train set : ' ,RV)

In [ ]:
test.groupby(['registered_via']).size()

In [ ]:
test['registered_via'] = test['registered_via'].replace(16, 3)

In [ ]:
test.groupby(['source_screen_name']).size()

In [ ]:
test['source_screen_name'] = test['source_screen_name'].replace('People global', 'Local playlist more')
test['source_screen_name'] = test['source_screen_name'].replace('People local', 'Local playlist more')

In [ ]:
test.groupby(['source_screen_name']).size()

### Feature Engineering

In [ ]:
def msno_count(df):
    df1 = df.groupby('msno').size()
    df1= df1.reset_index()
    df1.columns = ['msno', '#listened']
    return df1

# song_id, song_length
def mean_count_var1(df, var):
    df1 = df[[var, 'msno', 'target']].groupby([var]).agg(['mean','count'])
    df1 = df1.reset_index()
    df1.columns = [var,'mean', 'count']
    return df1

def mean_count_var2(df, var):
    df1 = df[[var, 'msno']].groupby([var]).agg(['count'])
    df1 = df1.reset_index()
    df1.columns = [var,'count']
    return df1

# genre_ids, lyricist, composer, artist_name
def player_preference(df, var, preference_by):
    df1 = df[['msno', var]].groupby(['msno', var]).size()
    df1 = df1.reset_index()
    df1.columns = ['msno', var, preference_by ]
    return df1
    #df1 = pd.merge(train, df, on = ['msno', var], how='left')

#몇명의 작곡가가 만들었나
#genre_ids, composer
def count_vals(x):
    if type(x) != str:
        return 1
    else:
        return 1 + x.count('|')


In [ ]:
data_sum = [test, train]

In [ ]:
train['number_of_genre'] =  train['genre_ids'].apply(count_vals)
train['number_of_composer'] =  train['composer'].apply(count_vals)

test['number_of_genre'] =  test['genre_ids'].apply(count_vals)
test['number_of_composer'] =  test['composer'].apply(count_vals)

In [ ]:
test = pd.merge(test,msno_count(test), on = 'msno' )
train = pd.merge(train, msno_count(train), on = 'msno' )

In [ ]:
train = train[train['number_of_composer'] < 30]
train = train[train['bd'] > -1]
train = train[train['bd'] < 70]

In [ ]:
lis = ['song_id', 'genre_ids', 'artist_name', 'composer', 'lyricist', 'language', 'city',]
for x in lis:
    train = pd.merge(train, mean_count_var1(train, x), on = x)
    test = pd.merge(test, mean_count_var2(test, x), on = x)
lis1 = ['song_id', 'genre_ids', 'artist_name', 'composer', 'lyricist']
lis2 = ['by_song', 'by_genre', 'by_artist', 'by_composer', 'by_lyricist']
for col, name in zip(lis1, lis2):
    train = pd.merge(train,  player_preference(train, col, name), on = ['msno', col], how = 'left')
    test = pd.merge(test,  player_preference(test, col, name), on = ['msno', col], how = 'left')

In [ ]:
train.to_csv('train1', encoding = 'utf8', index = False)
test.to_csv('test1', encoding = 'utf8', index = False )

In [43]:
train = pd.read_csv('train1', dtype = {'language' : 'category'})
test = pd.read_csv('test1', dtype = {'language' : 'category'})

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
lis = ['expiration_date', 'registration_init_time']
for x in lis:
    del train[x]
    del test[x]

In [46]:
train.columns

Index(['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'target', 'song_length', 'genre_ids', 'artist_name',
       'composer', 'lyricist', 'language', 'city', 'bd', 'gender',
       'registered_via', 'number_of_genre', 'number_of_composer', '#listened',
       'mean_x', 'count_x', 'mean_y', 'count_y', 'mean_x.1', 'count_x.1',
       'mean_y.1', 'count_y.1', 'mean_x.2', 'count_x.2', 'mean_y.2',
       'count_y.2', 'mean', 'count', 'by_song', 'by_genre', 'by_artist',
       'by_composer', 'by_lyricist'],
      dtype='object')

In [47]:
train.columns =['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'target', 'song_length', 'genre_ids', 'artist_name',
       'composer', 'lyricist', 'language', 'city', 'bd', 'gender',
       'registered_via', 'number_of_genre', 'number_of_composer', '#listened',
       'mean_1', 'count_1', 'mean_2', 'count_2', 'mean_3', 'count_3', 'mean_4',
       'count_4', 'mean_5', 'count_5', 'mean_6', 'count_6', 'mean_7', 'count_7',
       'by_song', 'by_genre', 'by_artist', 'by_composer', 'by_lyricist']

In [49]:
test.columns = ['id', 'msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'song_length', 'genre_ids', 'artist_name', 'composer',
       'lyricist', 'language', 'city', 'bd', 'gender', 'registered_via',
       'number_of_genre', 'number_of_composer', '#listened', 'count_1',
       'count_2', 'count_3', 'count_4', 'count_5', 'count_6', 'count_7',
       'by_song', 'by_genre', 'by_artist', 'by_composer', 'by_lyricist']

In [50]:
lis = ['song_id', 'genre_ids', 'artist_name', 'composer', 'lyricist', 'language', 'city']
lis2 = ['mean_1','mean_2','mean_3','mean_4','mean_5','mean_6','mean_7',]

for col, mean_name in zip(lis, lis2):
    df = train[[col, mean_name]].groupby(col).mean()
    df = df.reset_index()
    df.columns = [col, mean_name]
    test = pd.merge(test, df, on = col)

In [52]:
test=test[['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'song_length', 'genre_ids', 'artist_name',
       'composer', 'lyricist', 'language', 'city', 'bd', 'gender',
       'registered_via', 'number_of_genre', 'number_of_composer', '#listened',
       'mean_1', 'count_1', 'mean_2', 'count_2', 'mean_3', 'count_3', 'mean_4',
       'count_4', 'mean_5', 'count_5', 'mean_6', 'count_6', 'mean_7', 'count_7',
       'by_song', 'by_genre', 'by_artist', 'by_composer', 'by_lyricist']]

In [53]:
def to_cat(df):
    source_system_tab = pd.get_dummies(df['source_system_tab'], drop_first = True)
    source_screen_name = pd.get_dummies(df['source_screen_name'], drop_first = True)
    source_type = pd.get_dummies(df['source_type'], drop_first = True)
    city = pd.get_dummies(df['city'], drop_first = True)
    gender = pd.get_dummies(df['gender'], drop_first = True)
    registered_via = pd.get_dummies(df['registered_via'], drop_first = True)
    language = pd.get_dummies(df['language'], drop_first = True)

    frame = [source_system_tab , source_screen_name, source_type, city, gender, registered_via, language]
    df1 = pd.concat(frame, axis = 1)
    return df1

In [54]:
test_cat = to_cat(test)
train_cat = to_cat(train)

In [ ]:
train.columns

In [55]:
test = test[['song_length', 'bd', 'number_of_genre', 'number_of_composer', '#listened',
       'mean_1', 'count_1', 'mean_2', 'count_2', 'mean_3', 'count_3', 'mean_4',
       'count_4', 'mean_5', 'count_5', 'mean_6', 'count_6', 'mean_7', 'count_7',
       'by_song', 'by_genre', 'by_artist', 'by_composer', 'by_lyricist']]

In [56]:
train = train[['target', 'song_length', 'bd', 'number_of_genre', 'number_of_composer', '#listened',
       'mean_1', 'count_1', 'mean_2', 'count_2', 'mean_3', 'count_3', 'mean_4',
       'count_4', 'mean_5', 'count_5', 'mean_6', 'count_6', 'mean_7', 'count_7',
       'by_song', 'by_genre', 'by_artist', 'by_composer', 'by_lyricist']]

In [57]:
test = pd.concat([test, test_cat], axis = 1)
train = pd.concat([train, train_cat], axis = 1)

In [58]:
for x in list(train.columns):
    if x not in list(test.columns):
        print(x)

target


In [59]:
for x in list(test.columns):
    if x not in list(train.columns):
        print(x)

In [60]:
test.to_csv('test_final', encoding = 'utf-8', index=False)

In [61]:
train.to_csv('train_final', encoding = 'utf-8', index=False)

## Create Train Data Set and Evaluation(test) Data Set

In [ ]:
test = pd.read_csv('test_final')
train = pd.read_csv('train_final')

In [62]:
print(len(train.iloc[:, 1:].columns),
len(test.columns))

101 101


In [63]:
from sklearn.preprocessing import MinMaxScaler

In [65]:
scaler  = MinMaxScaler(copy=True, feature_range=(0, 1))
scaler.fit(train.iloc[:, 1:])
df_train = scaler.transform(train.iloc[:, 1:])
df_test = scaler.transform(test)

In [66]:
df_train1 = np.insert(df_train, 101, np.array(train.iloc[:,0]), axis=1)

In [67]:
df_train1_pd = pd.DataFrame(df_train1)
df_test1_pd = pd.DataFrame(df_test)

In [68]:
df_train1_pd.to_csv('XY', index = False, header=False)
df_test1_pd.to_csv('XY_test', index = False, header=False)

In [2]:
XY = pd.read_csv('XY')

In [3]:
lis = []
for y in XY.iloc[:, -1]:
    if y == 1:
        lis.append(0.0)
    else:
        lis.append(1.0)
        

In [4]:
lis = pd.DataFrame(lis)
XY = pd.merge(XY, lis, left_index = True, right_index = True)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(XY, XY.iloc[:, -1], test_size=0.30)

In [6]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [7]:
X_train.to_csv('X_train', index = False, header=False)
X_test.to_csv('X_test', index = False, header=False)